In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import os
import torch.optim as optim
import numpy as np
import random

import copy
import torch.optim as optim
# import timm


import copy
import math
import random
import time
from collections import OrderedDict, defaultdict
from typing import Union, List

import numpy as np
import torch
from matplotlib import pyplot as plt
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
# from torchprofile import profile_macs
from torchvision.datasets import *
from torchvision.transforms import *
from tqdm.auto import tqdm

# from torchprofile import profile_macs

# assert torch.cuda.is_available(), \
# "The current runtime does not have CUDA support." \
# "Please go to menu bar (Runtime - Change runtime type) and select GPU"


# import timm

import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
# !pip uninstall sconce
# !pip install git+https://github.com/satabios/sconce --upgrade

In [2]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 4, 3)
        self.bn1 = nn.BatchNorm2d(4)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(4, 6, 3)
        self.bn2 = nn.BatchNorm2d(6)
        self.fc1 = nn.Linear(6*6*6, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.pool(self.bn1(F.relu(self.conv1(x))))
        x = self.pool(self.bn2(F.relu(self.conv2(x))))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x



In [3]:

# from torchvision.transforms import Compose
image_size = 32
transforms = {
    "train": transforms.Compose([
        RandomCrop(image_size, padding=4),
        RandomHorizontalFlip(),
        ToTensor(),
    ]),
    "test": ToTensor(),
}
dataset = {}
for split in ["train", "test"]:
  dataset[split] = CIFAR10(
    root="data/cifar10",
    train=(split == "train"),
    download=True,
    transform=transforms[split],
  )
dataloader = {}
for split in ['train', 'test']:
  dataloader[split] = DataLoader(
    dataset[split],
    batch_size=512,
    shuffle=(split == 'train'),
    num_workers=0,
    pin_memory=True,
  )



Files already downloaded and verified
Files already downloaded and verified


In [4]:
for name, _ in Net().named_children():
    print(name)

conv1
bn1
pool
conv2
bn2
fc1
fc2


In [5]:
from sconce import sconce


sconces = sconce()
sconces.model= Net()
sconces.criterion = nn.CrossEntropyLoss()
sconces.optimizer= optim.Adam(sconces.model.parameters(), lr=1e-4)
sconces.scheduler= optim.lr_scheduler.CosineAnnealingLR(sconces.optimizer, T_max=200)
sconces.dataloader = dataloader

ModuleNotFoundError: No module named 'fast_pytorch_kmeans'

In [ ]:
sconces.bitwidth = 4
sconces.test_quant()